<a href="https://colab.research.google.com/github/nmuraro/entregables/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [ ]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [ ]:
df_anio_list = [
    pd.read_csv(url).rename(columns=str.lower)
    for url in archivos_anio
]
df_anio = pd.concat(df_anio_list, ignore_index=True)
print(df_anio.head())

# Buscar códigos ISO duplicados
duplicados = df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
print(duplicados)

# Eliminamos el registro no deseado (Burma)
df_codigos = df_codigos[df_codigos['pais'] != 'malo']

duplicados = df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
print(duplicados)

# Hacer la unión
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')

# Mostrar el resultado
print(df.head())

  codigo_iso  anio  indice  ranking
0        AFG  2001    35.5     59.0
1        AGO  2001    30.2     50.0
2        ALB  2001     NaN      NaN
3        AND  2001     NaN      NaN
4        ARE  2001     NaN      NaN
Empty DataFrame
Columns: [codigo_iso, pais]
Index: []
Empty DataFrame
Columns: [codigo_iso, pais]
Index: []
  codigo_iso  anio  indice  ranking                    pais
0        AFG  2001    35.5     59.0             Afghanistán
1        AGO  2001    30.2     50.0                  Angola
2        ALB  2001     NaN      NaN                 Albania
3        AND  2001     NaN      NaN                 Andorra
4        ARE  2001     NaN      NaN  Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [ ]:
#ESTRUCTURA DE DATOS

#¿Cuántas filas (observaciones) contiene el conjunto de datos?
#¿Cuántas columnas tiene el DataFrame?
# Número de filas y columnas
print("Shape (filas, columnas):", df.shape)

#¿Cuáles son los nombres de las columnas?
# Nombres de columnas
print("Nombres de columnas:", df.columns.tolist())

#¿Qué tipo de datos tiene cada columna?
# Tipos de datos
print("\nTipos de datos por columna:")
print(df.dtypes)
#¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?
# Información general
print("\nResumen con .info():")
df.info()

Shape (filas, columnas): (3060, 5)
Nombres de columnas: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']

Tipos de datos por columna:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object

Resumen con .info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


In [ ]:
# RESUMEN ESTADISTICO

#¿Qué observas sobre los valores de indice y ranking?
print("\nResumen estadístico:")
print(df.describe())

#¿Qué valores mínimo, máximo y promedio tiene la columna indice?
min_indice = df['indice'].min()
max_indice = df['indice'].max()
mean_indice = df['indice'].mean()
print(f"\nÍndice - Mínimo: {min_indice}, Máximo: {max_indice}, Promedio: {mean_indice:.2f}")
#¿Qué países presentan los valores extremos en indice y ranking?

# País con menor índice
print("\nPaises con menor índice:")
print(df[df['indice'] == min_indice][['pais', 'codigo_iso', 'anio', 'indice']])

# País con mayor índice
print("\nPaís con mayor índice:")
print(df[df['indice'] == max_indice][['pais', 'codigo_iso', 'anio', 'indice']])

# País con ranking 1 (más alto)
print("\nPaises con ranking más alto (ranking == 1):")
print(df[df['ranking'] == 1][['pais', 'codigo_iso', 'anio', 'ranking']])


Resumen estadístico:
              anio        indice        ranking
count  3060.000000   2664.000000    2837.000000
mean   2009.941176    205.782316     477.930913
std       5.786024   2695.525264    6474.935347
min    2001.000000      0.000000       1.000000
25%    2005.000000     15.295000      34.000000
50%    2009.000000     28.000000      70.000000
75%    2015.000000     41.227500     110.000000
max    2019.000000  64536.000000  121056.000000

Índice - Mínimo: 0.0, Máximo: 64536.0, Promedio: 205.78

Paises con menor índice:
              pais codigo_iso  anio  indice
1304     Dinamarca        DNK  2008     0.0
1313     Finlandia        FIN  2008     0.0
1335       Irlanda        IRL  2008     0.0
1382       Noruega        NOR  2008     0.0
1412        Suecia        SWE  2008     0.0
1468         Suiza        CHE  2009     0.0
1493     Finlandia        FIN  2009     0.0
1518      Islandia        ISL  2009     0.0
1561  Países Bajos        NLD  2009     0.0
1562       Noruega     

In [ ]:

#DATOS FALTANTES

#¿Cuántos valores nulos hay en cada columna?
# Cantidad de valores nulos por columna
print("\nValores nulos por columna:")
print(df.isnull().sum())


#¿Qué proporción de observaciones tienen valores faltantes?
# Proporción de valores faltantes por columna
print("\nProporción de valores nulos (%):")
print(df.isnull().mean() * 100)

#¿Hay columnas con más del 30% de datos faltantes?
# Filas con al menos un nulo
total_filas = len(df)
filas_con_nulos = df.isnull().any(axis=1).sum()
proporcion_filas_con_nulos = filas_con_nulos / total_filas * 100
print(f"\nFilas con al menos un valor nulo: {filas_con_nulos} ({proporcion_filas_con_nulos:.2f}%)")

# Columnas con más del 30% de valores nulos
print("\nColumnas con más del 30% de nulos:")
print(df.isnull().mean()[df.isnull().mean() > 0.3])



Valores nulos por columna:
codigo_iso      0
anio            0
indice        396
ranking       223
pais            0
dtype: int64

Proporción de valores nulos (%):
codigo_iso     0.000000
anio           0.000000
indice        12.941176
ranking        7.287582
pais           0.000000
dtype: float64

Filas con al menos un valor nulo: 397 (12.97%)

Columnas con más del 30% de nulos:
Series([], dtype: float64)


In [ ]:
#UNICIDAD Y DUPLICADOS

#¿Cuántos países distintos (pais) hay en el DataFrame?
# Países distintos
print("\nCantidad de países distintos:", df['pais'].nunique())

#¿Cuántos años distintos (anio) hay representados?
# Años distintos
print("Cantidad de años distintos:", df['anio'].nunique())

#¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?
# Filas duplicadas
duplicados = df.duplicated().sum()
print(f"Cantidad de filas duplicadas: {duplicados}")



Cantidad de países distintos: 179
Cantidad de años distintos: 17
Cantidad de filas duplicadas: 0


In [ ]:
#Validación cruzada de columnas
#¿Hay inconsistencias entre el país (pais) y su código (codigo_iso)? (por ejemplo, un mismo código ISO asociado a más de un país)
#Sugerencia: Apoya tu análisis con funciones como .info(), .nunique(), .isnull().sum(), .duplicated(), .value_counts(), entre otras.
# ¿Un código ISO aparece asociado a más de un nombre de país?
duplicidad_codigo_iso = df.groupby('codigo_iso')['pais'].nunique()
duplicados = duplicidad_codigo_iso[duplicidad_codigo_iso > 1]
print("\nCódigos ISO asociados a más de un país (inconsistencias):")
print(duplicados)



Códigos ISO asociados a más de un país (inconsistencias):
Series([], Name: pais, dtype: int64)





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [ ]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america =  pd.DataFrame() # FIX ME
df_america = df[df['codigo_iso'].isin(america)].copy()

print(df_america[['anio', 'pais', 'codigo_iso']].head())


    anio               pais codigo_iso
5   2001          Argentina        ARG
7   2001  Antigua y Barbuda        ATG
20  2001             Belize        BLZ
21  2001            Bolivia        BOL
22  2001             Brasil        BRA


In [ ]:
#a)
#El país con el menor valor de indice (mayor libertad de prensa).
#El país con el mayor valor de indice (menor libertad de prensa).
anios = sorted(df_america['anio'].unique())

print("Países con mayor y menor libertad de prensa por año (FOR LOOP):\n")
for anio in anios:
    df_anio = df_america[df_america['anio'] == anio]

    # Omitimos el año si todos los valores son NaN
    if df_anio['indice'].isnull().all():
        continue

    # Mínimo índice (mayor libertad)
    min_ind = df_anio['indice'].min()
    pais_min = df_anio[df_anio['indice'] == min_ind]['pais'].values[0]

    # Máximo índice (menor libertad)
    max_ind = df_anio['indice'].max()
    pais_max = df_anio[df_anio['indice'] == max_ind]['pais'].values[0]

    print(f"Año {anio}:")
    print(f"  ✅ Mayor libertad: {pais_min} ({min_ind})")
    print(f"  ❌ Menor libertad: {pais_max} ({max_ind})\n")


Países con mayor y menor libertad de prensa por año (FOR LOOP):

Año 2001:
  ✅ Mayor libertad: Canadá (0.8)
  ❌ Menor libertad: Cuba (90.3)

Año 2002:
  ✅ Mayor libertad: Trinidad y Tobago (1.0)
  ❌ Menor libertad: Cuba (97.83)

Año 2003:
  ✅ Mayor libertad: Trinidad y Tobago (2.0)
  ❌ Menor libertad: Argentina (35826.0)

Año 2004:
  ✅ Mayor libertad: Trinidad y Tobago (2.0)
  ❌ Menor libertad: Cuba (87.0)

Año 2005:
  ✅ Mayor libertad: Bolivia (4.5)
  ❌ Menor libertad: Cuba (95.0)

Año 2006:
  ✅ Mayor libertad: Canadá (4.88)
  ❌ Menor libertad: Cuba (96.17)

Año 2007:
  ✅ Mayor libertad: Canadá (3.33)
  ❌ Menor libertad: Cuba (88.33)

Año 2008:
  ✅ Mayor libertad: Canadá (3.7)
  ❌ Menor libertad: Cuba (94.0)

Año 2009:
  ✅ Mayor libertad: Estados Unidos (6.75)
  ❌ Menor libertad: Cuba (78.0)

Año 2012:
  ✅ Mayor libertad: Jamaica (9.88)
  ❌ Menor libertad: Cuba (71.64)

Año 2013:
  ✅ Mayor libertad: Jamaica (10.9)
  ❌ Menor libertad: Cuba (70.92)

Año 2014:
  ✅ Mayor libertad: Canadá 

In [ ]:
#b) Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con groupby, sin usar ciclos explícitos.

# Eliminar filas con índice NaN
df_america_valid = df_america.dropna(subset=['indice'])

# Obtener el índice mínimo por año
idx_min = df_america_valid.groupby('anio')['indice'].idxmin()
min_por_anio = df_america_valid.loc[idx_min][['anio', 'pais', 'indice']].rename(columns={'pais': 'mayor_libertad', 'indice': 'indice_min'})

# Obtener el índice máximo por año
idx_max = df_america_valid.groupby('anio')['indice'].idxmax()
max_por_anio = df_america_valid.loc[idx_max][['anio', 'pais', 'indice']].rename(columns={'pais': 'menor_libertad', 'indice': 'indice_max'})

# Combinar ambos resultados
extremos = pd.merge(min_por_anio, max_por_anio, on='anio')

# Mostrar
print("\nPaíses con mayor y menor libertad de prensa por año (VECTORIZADO):")
print(extremos.head())



Países con mayor y menor libertad de prensa por año (VECTORIZADO):
   anio     mayor_libertad  indice_min menor_libertad  indice_max
0  2001             Canadá         0.8           Cuba       90.30
1  2002  Trinidad y Tobago         1.0           Cuba       97.83
2  2003  Trinidad y Tobago         2.0      Argentina    35826.00
3  2004  Trinidad y Tobago         2.0           Cuba       87.00
4  2005            Bolivia         4.5           Cuba       95.00


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
#Construye una tabla dinámica (pivot_table) donde las filas correspondan a los países, las columnas a los años (anio) y los valores sean el indice máximo alcanzado por cada país en ese año.
#Asegúrate de reemplazar los valores nulos resultantes con 0.
#Hint: Puedes utilizar el parámetro fill_value=0 en pd.pivot_table(...).

tabla_indice = pd.pivot_table(
    df_america,
    index='pais',
    columns='anio',
    values='indice',
    aggfunc='max',
    fill_value=0
)
print(tabla_indice.head())

anio               2001   2002     2003   2004   2005   2006   2007   2008  \
pais                                                                         
Antigua y Barbuda   0.0   0.00      0.0   0.00   0.00   0.00   0.00   0.00   
Argentina          12.0  15.17  35826.0  13.67  17.30  24.83  14.08  11.33   
Belize              0.0   0.00      0.0   0.00   0.00   0.00   0.00   0.00   
Bolivia            14.5   9.67     20.0   9.67   4.50  21.50  28.20  24.17   
Brasil             18.8  16.75     16.5  14.50  17.17  25.25  18.00  15.88   

anio                2009   2012   2013   2014   2015   2017   2018   2019  
pais                                                                       
Antigua y Barbuda   0.00   0.00  20.81   0.00   0.00   0.00   0.00   0.00  
Argentina          16.35  25.67  25.27  26.11  25.09  25.07  26.05  28.30  
Belize              0.00   0.00  17.05  18.54  20.61  23.43  24.55  27.50  
Bolivia            28.13  32.80  31.04  31.29  31.78  33.88  32.45  35.38

In [ ]:
#a) ¿Qué país tiene el mayor valor de indice en toda la tabla resultante?
#¿Y cuál tiene el menor (distinto de cero)?

# Máximo valor absoluto y país correspondiente
valor_max = tabla_indice.max().max()
pais_max = tabla_indice[tabla_indice.max(axis=1) == valor_max].index[0]

# Mínimo valor (excluyendo ceros)
valor_min = tabla_indice.replace(0, np.nan).min().min()
pais_min = tabla_indice[tabla_indice.replace(0, np.nan).min(axis=1) == valor_min].index[0]

print(f"País con mayor índice: {pais_max} ({valor_max})")
print(f"País con menor índice distinto de cero: {pais_min} ({valor_min})")

#b) ¿Qué años presentan en promedio los valores de indice más altos? ¿Y los más bajos?

promedios_anio = tabla_indice.replace(0, np.nan).mean(axis=0)  # promedio por año, sin contar ceros

anio_mas_alto = promedios_anio.idxmax()
anio_mas_bajo = promedios_anio.idxmin()

print(f"Año con mayor promedio de índice: {anio_mas_alto} ({promedios_anio[anio_mas_alto]:.2f})")
print(f"Año con menor promedio de índice: {anio_mas_bajo} ({promedios_anio[anio_mas_bajo]:.2f})")


País con mayor índice: Argentina (35826.0)
País con menor índice distinto de cero: Canadá (0.8)
Año con mayor promedio de índice: 2003 (1451.94)
Año con menor promedio de índice: 2002 (16.75)


In [ ]:
#c) ¿Qué país muestra mayor variabilidad (diferencia entre su máximo y mínimo indice a lo largo del tiempo)?
tabla_sin_ceros = tabla_indice.replace(0, np.nan)
variabilidad = tabla_sin_ceros.max(axis=1) - tabla_sin_ceros.min(axis=1)

pais_mas_variable = variabilidad.idxmax()
print(f"📉 País con mayor variabilidad en índice: {pais_mas_variable} ({variabilidad[pais_mas_variable]:.2f})")


📉 País con mayor variabilidad en índice: Argentina (35814.67)


In [ ]:
#d) ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?
constantes = tabla_sin_ceros.dropna().nunique(axis=1) == 1
paises_constantes = constantes[constantes].index.tolist()

print("📌 Países con índice constante a lo largo del tiempo:")
print(paises_constantes if paises_constantes else "Ninguno")


📌 Países con índice constante a lo largo del tiempo:
Ninguno


In [ ]:
#e) ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?
sin_datos = tabla_indice.sum(axis=1) == 0
paises_sin_datos = tabla_indice[sin_datos].index.tolist()

print("❌ Países sin ningún dato de índice:")
print(paises_sin_datos if paises_sin_datos else "Ninguno")


❌ Países sin ningún dato de índice:
Ninguno
